<a href="https://colab.research.google.com/github/aarsanjani/meansquares/blob/master/version2/preprocessing/TX_MergecombinedDatasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal

This colab merges for TX
* COVID-19 and population datasets
* Social distancing data
* Mask rule active data
* Google mobility data 

# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021'

 CA_CovidDataJan10.csv		       NY_CovidDataJan10.csv
 CA_GoogleMobilityDataJan10.csv        NY_GoogleMobilityDataJan10.csv
 CA_SocialDistancingDataJan10.csv      NY_SocialDistancingDataJan10.csv
 computed_CA_mobility_data.csv	      'Policy_Map_(Sized)_Full_Data_data.csv'
 data-mask_for_all.csv		       TX_CovidDataJan10.csv
 election_results_2021.csv	       TX_GoogleMobilityDataJan10.csv
 maryland_inertia_raw_dec28_2020.csv   TX_SocialDistancingDataJan10.csv
 Mask_Rule_Jan10.csv


# Import package

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler as ss
import itertools
from scipy import linalg
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
import numpy as np
import matplotlib.pyplot as plt

# Load Data 

In [ ]:
TX_COVID19_data = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/TX_CovidDataJan10.csv',low_memory=False)
TX_SD_data =pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/TX_SocialDistancingDataJan10.csv', low_memory=False)
mask_rule_data = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/Mask_Rule_Jan10.csv', low_memory=False)
TX_google_mobility_data = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/TX_GoogleMobilityDataJan10.csv', low_memory=False)




In [ ]:
print(TX_COVID19_data.shape)
print(TX_SD_data.shape)
print(mask_rule_data.shape)
print(TX_google_mobility_data.shape)

(89662, 11)
(63301, 9)
(52, 6)
(60853, 14)


In [ ]:
mask_rule_data.head(2)

,state_name,Law Date,Masks Required?,Type of Requirement,mask_required_rule,requirement_code
0,Mississippi,2020-08-04,Parts of State,NaN,6,-1
1,Vermont,2020-08-01,Entire State,Everywhere in public where social distancing i...,0,5


In [ ]:
TX_mask_rule = mask_rule_data[mask_rule_data['state_name'] == 'Texas']
TX_mask_rule

,state_name,Law Date,Masks Required?,Type of Requirement,mask_required_rule,requirement_code
13,Texas,2020-07-03,Entire State,Everywhere in public where social distancing i...,0,6


In [ ]:
TX_COVID19_data.head()

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,County Name,State,population
0,2020-01-22,Texas,48001,0,0.0,0,0,Anderson County,TX,57735
1,2020-01-23,Texas,48001,0,0.0,0,0,Anderson County,TX,57735
2,2020-01-24,Texas,48001,0,0.0,0,0,Anderson County,TX,57735
3,2020-01-25,Texas,48001,0,0.0,0,0,Anderson County,TX,57735
4,2020-01-26,Texas,48001,0,0.0,0,0,Anderson County,TX,57735


In [ ]:
TX_COVID19_data['FIPS'] = TX_COVID19_data['FIPS'].apply(lambda x :int(x))
TX_COVID19_data.head()

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,County Name,State,population
0,2020-01-22,Texas,48001,0,0.0,0,0,Anderson County,TX,57735
1,2020-01-23,Texas,48001,0,0.0,0,0,Anderson County,TX,57735
2,2020-01-24,Texas,48001,0,0.0,0,0,Anderson County,TX,57735
3,2020-01-25,Texas,48001,0,0.0,0,0,Anderson County,TX,57735
4,2020-01-26,Texas,48001,0,0.0,0,0,Anderson County,TX,57735


# Combining COVID-19, population  and mask data

In [ ]:
TX_COVID19_data['mask_rule_active'] = TX_COVID19_data['Date'].apply(lambda x : 1 if x >= TX_mask_rule['Law Date'].values[0] else 0 )
TX_COVID19_data.head()

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,County Name,State,population,mask_rule_active
0,2020-01-22,Texas,48001,0,0.0,0,0,Anderson County,TX,57735,0
1,2020-01-23,Texas,48001,0,0.0,0,0,Anderson County,TX,57735,0
2,2020-01-24,Texas,48001,0,0.0,0,0,Anderson County,TX,57735,0
3,2020-01-25,Texas,48001,0,0.0,0,0,Anderson County,TX,57735,0
4,2020-01-26,Texas,48001,0,0.0,0,0,Anderson County,TX,57735,0


In [ ]:
TX_COVID19_data.shape

(89662, 11)

# Combining COVID-19, population, mask data and Social distancing data

In [ ]:
TX_SD_data.tail(2)

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
63299,2021-01-06,US,2,Texas,Zavala County,48507.0,332,3.502,36
63300,2021-01-07,US,2,Texas,Zavala County,48507.0,308,2.482,25


In [ ]:
TX_SD_data['fips'] = TX_SD_data['fips'].apply(lambda x :int(x))
TX_SD_data.head()

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
0,2020-03-01,US,1,Texas,NaN,48,633045,6.619,72
1,2020-03-02,US,1,Texas,NaN,48,666618,9.071,100
2,2020-03-03,US,1,Texas,NaN,48,677068,9.062,99
3,2020-03-04,US,1,Texas,NaN,48,693767,8.571,94
4,2020-03-05,US,1,Texas,NaN,48,665192,9.828,108


In [ ]:
TX_SD_data.tail(2)

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
63299,2021-01-06,US,2,Texas,Zavala County,48507,332,3.502,36
63300,2021-01-07,US,2,Texas,Zavala County,48507,308,2.482,25


In [ ]:
TX_socialDistancing_combined = pd.merge(TX_COVID19_data,TX_SD_data,how='inner',left_on=['Date','County Name'],right_on=['date','admin2'])
TX_socialDistancing_combined.head(2)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,County Name,State,population,mask_rule_active,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
0,2020-03-01,Texas,48001,0,0.0,0,0,Anderson County,TX,57735,0,2020-03-01,US,2,Texas,Anderson County,48001,1210,8.910,79
1,2020-03-02,Texas,48001,0,0.0,0,0,Anderson County,TX,57735,0,2020-03-02,US,2,Texas,Anderson County,48001,1222,10.708,95


In [ ]:
TX_socialDistancing_combined.shape

(62999, 20)

# Combining COVID-19, population, mask data, Social distancing data and Google Mobility data

In [ ]:
TX_google_mobility_data.tail(2)

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
60851,US,United States,Texas,Zavala County,NaN,NaN,48507.0,2021-01-04,NaN,NaN,NaN,NaN,-32.0,NaN
60852,US,United States,Texas,Zavala County,NaN,NaN,48507.0,2021-01-05,NaN,NaN,NaN,NaN,-31.0,NaN


In [ ]:
TX_all_data_merged = pd.merge(TX_socialDistancing_combined,TX_google_mobility_data,how='inner',left_on=['Date','County Name'],right_on=['date','sub_region_2'])
TX_all_data_merged.head(2)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,County Name,State,population,mask_rule_active,date_x,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date_y,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,2020-03-01,Texas,48001,0,0.0,0,0,Anderson County,TX,57735,0,2020-03-01,US,2,Texas,Anderson County,48001,1210,8.910,79,US,United States,Texas,Anderson County,NaN,NaN,48001.0,2020-03-01,16.0,15.0,NaN,NaN,3.0,NaN
1,2020-03-02,Texas,48001,0,0.0,0,0,Anderson County,TX,57735,0,2020-03-02,US,2,Texas,Anderson County,48001,1222,10.708,95,US,United States,Texas,Anderson County,NaN,NaN,48001.0,2020-03-02,14.0,7.0,NaN,NaN,2.0,-1.0


In [ ]:
TX_all_data_merged.shape

(53463, 34)

In [ ]:
TX_all_data_merged.columns

Index(['Date', 'Province_State', 'FIPS', 'Confirmed', 'Deaths', 'New cases',
       'New deaths', 'County Name', 'State', 'population', 'mask_rule_active',
       'date_x', 'country_code', 'admin_level', 'admin1', 'admin2', 'fips',
       'samples', 'm50', 'm50_index', 'country_region_code', 'country_region',
       'sub_region_1', 'sub_region_2', 'metro_area', 'iso_3166_2_code',
       'census_fips_code', 'date_y',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline'],
      dtype='object')

In [ ]:
required_cols = ['Date', 'Province_State', 'FIPS',  'New cases',
        'County Name', 'population', 'mask_rule_active',
        'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline']

In [ ]:
TX_all_data_selected = TX_all_data_merged[required_cols]
TX_all_data_selected.head(2)

,Date,Province_State,FIPS,New cases,County Name,population,mask_rule_active,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,2020-03-01,Texas,48001,0,Anderson County,57735,0,8.910,79,16.0,15.0,NaN,NaN,3.0,NaN
1,2020-03-02,Texas,48001,0,Anderson County,57735,0,10.708,95,14.0,7.0,NaN,NaN,2.0,-1.0


In [ ]:
TX_all_data_selected.fillna(0)

,Date,Province_State,FIPS,New cases,County Name,population,mask_rule_active,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,2020-03-01,Texas,48001,0,Anderson County,57735,0,8.910,79,16.0,15.0,0.0,0.0,3.0,0.0
1,2020-03-02,Texas,48001,0,Anderson County,57735,0,10.708,95,14.0,7.0,0.0,0.0,2.0,-1.0
2,2020-03-03,Texas,48001,0,Anderson County,57735,0,12.118,107,21.0,8.0,0.0,0.0,2.0,-2.0
3,2020-03-04,Texas,48001,0,Anderson County,57735,0,9.156,81,12.0,5.0,0.0,0.0,1.0,2.0
4,2020-03-05,Texas,48001,0,Anderson County,57735,0,11.230,100,18.0,13.0,0.0,0.0,3.0,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53458,2020-12-30,Texas,48507,10,Zavala County,11840,1,4.564,47,0.0,0.0,0.0,0.0,-38.0,0.0
53459,2020-12-31,Texas,48507,5,Zavala County,11840,1,3.274,34,0.0,0.0,0.0,0.0,-44.0,0.0
53460,2021-01-01,Texas,48507,0,Zavala County,11840,1,1.683,17,0.0,0.0,0.0,0.0,-61.0,0.0
53461,2021-01-04,Texas,48507,0,Zavala County,11840,1,10.211,106,0.0,0.0,0.0,0.0,-32.0,0.0


# Store file to Google Drive

In [ ]:
from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/'
filename = 'TX_alldatasets_'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
TX_all_data_selected.to_csv(location+filename,index=False)

TX_alldatasets_Jan11.csv
